In [735]:
import pandas as pd
from connect import PostgreSQL
from tqdm import tqdm
import datetime as dt
from datetime import date
import plotly.graph_objs as go

In [736]:
def proportion_size(pixels:int=None, size:int=None, padrao:int=2160):
    sizing = float(pixels / (padrao / size))
    return sizing

# Parâmetros

In [737]:
_schema = 'D-1'

In [738]:
_pixels = 872

pixels_width = int((_pixels/9)*16) + 45
pixels_height = int(_pixels) + 15

In [739]:
date_query = date(int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[0])), int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[1])), 1).strftime('%Y-%m-%d')

In [740]:
codigos_kaizen = [('KAIZEN ASA NORTE', '13996'),
                  ('KAIZEN CEILÂNDIA', '16100'),
                  (     'KAIZEN GAMA', '18400'),
                  (  'KAIZEN SOF SUL', '20000')]

In [741]:
lista_lojas = [('01',             'CD', '#FFFFFF'),
               ('02',               '', '#FFFFFF'),
               ('03',  '04. ASA NORTE', '#FFFFFF'),
               ('04',  '02. CEILÂNDIA', '#FFFFFF'),
               ('05',       '03. GAMA', '#FFFFFF'),
               ('06',    '01. SOF SUL', '#FFFFFF'),
               ('VO', '05. E-COMMERCE', '#FFFFFF')]

In [742]:
dias_semana = [(   'Sunday',       'Domingo'),
               (   'Monday', 'Segunda-feira'),
               (  'Tuesday',   'Terça-feira'),
               ('Wednesday',  'Quarta-feira'),
               ( 'Thursday',  'Quinta-feira'),
               (   'Friday',   'Sexta-feira'),
               ( 'Saturday',        'Sábado')]

In [743]:
relatorio = 'ACOMPANHAMENTO DE VENDA'

relatorio_lower = relatorio.lower()
relatorio_msg = relatorio_lower.replace(relatorio_lower.split()[0], relatorio_lower.split()[0].title())

In [744]:
caminho_imagem = 'Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\REPORT'

In [745]:
contatos = [('PrimeiraFoto', 'https://chat.whatsapp.com/LK4LDajkH52BNydoLGGD2d', True),
            ('Dados e Int',  'https://chat.whatsapp.com/JxY14kWarG70MxB2jzb5pc', False),
            ('Dream Team',   'https://chat.whatsapp.com/Hb0KMgYnuFB8coTPlFKTzJ', False),
            ('Report Di',    'https://chat.whatsapp.com/G4p8qdUIUblCNPS80244RQ', False),
            ('API',          'https://chat.whatsapp.com/EBLX0fxYfvuHL7jZ9CZECQ', True),
            ('Teste Python', 'https://chat.whatsapp.com/I4y7EAVfns629NvfoLUtGI', True)]

#

# DataFrames

## Clientes

In [746]:
df_cliente = PostgreSQL.read_postgres(name_table='cliente', schema=_schema)

In [747]:
df_cli_ped = PostgreSQL.read_postgres(name_table='cli_ped', schema=_schema, query=f'WHERE DT_CADASTR >= \'{date_query}\'')

## Pedidos

In [748]:
df_pedido = PostgreSQL.read_postgres(name_table='pedido', schema=_schema, query=f'WHERE DT_EMISSAO >= \'{date_query}\'')

In [749]:
df_pedido['VALOR_TOT'] = df_pedido['VALOR_TOT'].astype(float)

In [750]:
df_pedido = df_pedido[(df_pedido['CODCLI'] != '13996') &
                      (df_pedido['CODCLI'] != '16100') &
                      (df_pedido['CODCLI'] != '18400') &
                      (df_pedido['CODCLI'] != '20000') &
                      (df_pedido['CODVDE'] != '0100') &
                      (df_pedido['CODVDE'] != '0001') &
                      (df_pedido['TIPPED'] == 'V')]

In [751]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'FORMA_PGTO'] == 'M':
        df_pedido.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_pedido.loc[i, 'COD_LOJA'] = df_pedido.loc[i, 'CD_LOJA']

100%|██████████| 5408/5408 [00:00<00:00, 11569.79it/s]


In [752]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        df_pedido.loc[i, 'NU_CPFCNPJ'] = list(df_cli_ped[(df_cli_ped['CD_LOJA'] == df_pedido.loc[i, 'CD_LOJA']) &
                                             (df_cli_ped['SG_PEDIDO'] == df_pedido.loc[i, 'SERIE']) &
                                             (df_cli_ped['NU_PEDIDO'] == df_pedido.loc[i, 'NU_NOTA'])]['NU_CPFCNPJ'])[0]

100%|██████████| 5408/5408 [00:00<00:00, 36020.50it/s]


In [753]:
df_pedido = pd.merge(df_pedido, df_cliente, how='inner', on='CODCLI')

In [754]:
for i in tqdm(df_pedido.index):

    if df_pedido.loc[i, 'COD_LOJA'] == 'VO':
        identify = df_pedido.loc[i, 'NU_CPFCNPJ']
    else:
        identify = df_pedido.loc[i, 'CPF_CGC']

    if df_pedido.loc[i, 'CODAREA'] == '104':
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_pedido.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

100%|██████████| 5408/5408 [00:00<00:00, 9495.74it/s]


## Devolução

In [755]:
df_prod_ent = PostgreSQL.read_postgres(full_query=f'select a.*, (a.VL_PRECO*a.QT_DEVOLVE) as VL_TOTAL_DEV, b.IN_CLIFOR as IN_CLIFOR1, b.IN_CANCELA, UPPER(b.NFEENVSTAT) as NFEENVSTAT, c.FORMA_PGTO from "{_schema}".prod_ent a inner join "{_schema}".entrada b on a.CD_LOJA = b.CD_LOJA and a.SG_SERIE = b.SG_SERIE and a.NU_NOTA = b.NU_NOTA inner join "{_schema}".venda c on a.CD_LOJA = c.CD_LOJA and a.SG_ORIGEM = c.SERIE and a.NU_ORIGEM = c.NU_NOTA where a.DT_EMISSAO >= \'{date_query}\'')

In [756]:
df_prod_ent = df_prod_ent[(df_prod_ent['IN_CANCELA'] == 'N') &
                          (df_prod_ent['IN_CLIFOR1'] == 'C') &
                          (df_prod_ent['CD_CFOP'] != '1949') &
                          (df_prod_ent['CD_CFOP'] != '2949') &
                          (df_prod_ent['CD_CFOP'] != '1603') &
                          (df_prod_ent['CD_CLIENTE'] != '13996') &
                          (df_prod_ent['CD_CLIENTE'] != '16100') &
                          (df_prod_ent['CD_CLIENTE'] != '18400') &
                          (df_prod_ent['CD_CLIENTE'] != '20000') &
                          (df_prod_ent['NFEENVSTAT'].str.contains('DENEG') != True)]

In [757]:
for i in tqdm(df_prod_ent.index):
    if df_prod_ent.loc[i, 'FORMA_PGTO'] == 'M':
        df_prod_ent.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_prod_ent.loc[i, 'COD_LOJA'] = df_prod_ent.loc[i, 'CD_LOJA']

100%|██████████| 448/448 [00:00<00:00, 10172.74it/s]


In [758]:
df_prod_ent['VL_TOTAL_DEV'] = df_prod_ent['VL_TOTAL_DEV'].astype(float)

In [759]:
df_prod_ent = df_prod_ent.rename(columns={'CD_CLIENTE':'CODCLI'})
df_prod_ent = pd.merge(df_prod_ent, df_cliente, how='inner', on='CODCLI')

In [760]:
for i in tqdm(df_prod_ent.index):

    identify = df_prod_ent.loc[i, 'CPF_CGC']

    if df_prod_ent.loc[i, 'CODAREA'] == '104':
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ*'
    elif len(identify) < 14:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CPF'
    else:
        df_prod_ent.loc[i, 'TIPO_CLIENTE'] = 'CNPJ'

100%|██████████| 448/448 [00:00<00:00, 10916.96it/s]


## Metas

In [761]:
df_metas = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Metas')

In [762]:
df_data = pd.read_excel('Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\PROJETOS\\EXCEL\\BASES\\DATAS.xlsx', sheet_name='Datas')

In [763]:
df_datas = df_data[df_data['Mês - Ano'] == df_data[df_data['Data'] == date_query]['Mês - Ano'].values[0]].reset_index(drop=True)

In [764]:
df_meta_mes = df_metas[df_metas['Mês - Ano'] == df_datas['Mês - Ano'][0]]

In [765]:
df_meta_data = df_datas.drop(columns=['Dia', 'Mês', 'Ano', 'Dia da Semana', 'Nome - Mês', 'Semestre', 'Trimestre', 'Sequencial - Dia da Semana', 'Dias Úteis']).merge(df_meta_mes.drop(columns=['Mês', 'Ano', 'Dias Úteis', 'Qtd Sábados', 'Nome - Mês', 'Loja']), how='inner', on='Mês - Ano')

In [766]:
for i in df_meta_data.index:
    if df_meta_data.loc[i, 'Dia Útil'] == 'Sim':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Diária']
    
    elif df_meta_data.loc[i, 'Dia Útil'] == 'Meio':
        df_meta_data.loc[i, 'Meta'] = df_meta_data.loc[i, 'Meta Sábado']

    else:
        df_meta_data.loc[i, 'Meta'] = 0

df_meta_data = df_meta_data.rename(columns={'Cd_Loja':'CD_LOJA', 'Data':'DT_EMISSAO'})

# Cálculo

In [767]:
df_venda_bruta = df_pedido.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\464804912.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [768]:
df_devolucao = df_prod_ent.groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\793352684.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [769]:
df_venda_bruta_cnpj = df_pedido[df_pedido['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
df_devolucao_cnpj = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] != 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\3541458602.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\3541458602.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [770]:
df_venda_bruta_cpf = df_pedido[df_pedido['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VALOR_TOT']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})
df_devolucao_cpf = df_prod_ent[df_prod_ent['TIPO_CLIENTE'] == 'CPF'].groupby(['COD_LOJA', 'DT_EMISSAO']).sum()[['VL_TOTAL_DEV']].reset_index().rename(columns={'COD_LOJA':'CD_LOJA'})

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\4099021309.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\admin\AppData\Local\Temp\ipykernel_8368\4099021309.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [771]:
df_final = pd.merge(df_venda_bruta, df_devolucao, how='outer', on=['CD_LOJA', 'DT_EMISSAO'])

df_final = df_final.sort_values(by=['DT_EMISSAO', 'CD_LOJA'])
df_final = df_final.reset_index(drop=True)

df_final = pd.merge(df_meta_data, df_final, how='left', on=['CD_LOJA', 'DT_EMISSAO'])

In [772]:
df_final = df_final.fillna(0)

In [773]:
df_hoje = df_final
df_hoje = df_hoje.rename(columns={'VALOR_TOT':'VENDA BRUTA', 'VL_TOTAL_DEV':'DEVOLUÇÃO', 'Meta':'META'})

for i in tqdm(df_hoje.index):
    df_hoje.loc[i, 'VENDA LÍQUIDA'] = (df_hoje.loc[i, 'VENDA BRUTA'] - df_hoje.loc[i, 'DEVOLUÇÃO'])
    
    client_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] != 'CPF')]['CPF_CGC'].drop_duplicates())

    client_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                     (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                     (df_pedido['TIPO_CLIENTE'] == 'CPF')]['CPF_CGC'])

    venda_cgc = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_pedido['TIPO_CLIENTE'] != 'CPF')]['VALOR_TOT'])

    venda_cpf = list(df_pedido[(df_pedido['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_pedido['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_pedido['TIPO_CLIENTE'] == 'CPF')]['VALOR_TOT'])

    devol_cgc = list(df_prod_ent[(df_prod_ent['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_prod_ent['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_prod_ent['TIPO_CLIENTE'] != 'CPF')]['VL_TOTAL_DEV'])

    devol_cpf = list(df_prod_ent[(df_prod_ent['DT_EMISSAO'] == df_hoje.loc[i, 'DT_EMISSAO']) &
                    (df_prod_ent['COD_LOJA'] == df_hoje.loc[i, 'CD_LOJA']) &
                    (df_prod_ent['TIPO_CLIENTE'] == 'CPF')]['VL_TOTAL_DEV'])

    if sum(devol_cgc) > 0 and sum(venda_cgc) > 0:
        percent_cgc = sum(devol_cgc) / sum(venda_cgc)
    else:
        percent_cgc = 0

    if sum(devol_cpf) > 0 and sum(venda_cpf) > 0:
        percent_cpf = sum(devol_cpf) / sum(venda_cpf)
    else:
        percent_cpf = 0

    df_hoje.loc[i, 'VENDA_CNPJ'] = (sum(venda_cgc) - sum(devol_cgc))
    df_hoje.loc[i, 'DEVOL_CNPJ'] = sum(devol_cgc)
    df_hoje.loc[i, '% DEV CNPJ'] = percent_cgc
    df_hoje.loc[i, 'CNPJ'] = len(client_cgc)

    df_hoje.loc[i, 'VENDA_CPF'] = (sum(venda_cpf) - sum(devol_cpf))
    df_hoje.loc[i, 'DEVOL_CPF'] = sum(devol_cpf)
    df_hoje.loc[i, '% DEV CPF'] = percent_cpf
    df_hoje.loc[i, 'CPF'] = len(client_cpf)

    if df_hoje.loc[i, 'VENDA LÍQUIDA'] == 0:
        df_hoje.loc[i, '% META DIA'] = 0
    else:
        df_hoje.loc[i, '% META DIA'] = df_hoje.loc[i, 'VENDA LÍQUIDA'] / df_hoje.loc[i, 'META']

    if df_hoje.loc[i, 'DEVOLUÇÃO'] == 0:
        df_hoje.loc[i, '% DEV'] = 0
    else:
        df_hoje.loc[i, '% DEV'] = df_hoje.loc[i, 'DEVOLUÇÃO'] / df_hoje.loc[i, 'VENDA BRUTA']

  6%|▌         | 9/155 [00:00<00:01, 87.30it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_8368\774047444.py:54: RuntimeWarning:

divide by zero encountered in double_scalars

100%|██████████| 155/155 [00:01<00:00, 145.27it/s]


In [774]:
df_hoje = df_hoje.fillna(0)

# Vendas Online

In [785]:
df_print_vo = df_hoje[df_hoje['CD_LOJA'] == 'VO'].reset_index(drop=True)
df_print_vo = df_print_vo[['DT_EMISSAO',
                           'Nome - Dia da Semana',
                           'CD_LOJA',
                           'META',
                           'VENDA BRUTA',
                           'VENDA LÍQUIDA',
                           'DEVOLUÇÃO',
                           '% DEV',
                           'VENDA_CNPJ',
                           'DEVOL_CNPJ',
                           '% DEV CNPJ',
                           'CNPJ',
                           'VENDA_CPF',
                           'DEVOL_CPF',
                           '% DEV CPF',
                           'CPF',
                           '% META DIA']]

venda_mtd = []
meta_mtd = []

for i in df_print_vo.index:

    venda_mtd.append(list(df_print_vo.loc[:i, 'VENDA LÍQUIDA']))
    df_print_vo.loc[i, 'VENDA_MTD'] = sum(venda_mtd[i])

    meta_mtd.append(list(df_print_vo.loc[:i, 'META']))
    df_print_vo.loc[i, 'META_MTD'] = sum(meta_mtd[i])

    if df_print_vo.loc[i, 'VENDA_MTD'] == 0:
        df_print_vo.loc[i, '% META MÊS'] = 0
    else:
        df_print_vo.loc[i, '% META MÊS'] = df_print_vo.loc[i, 'VENDA_MTD'] / df_print_vo.loc[i, 'META_MTD']






for i in df_print_vo.index:

    df_print_vo.loc[i, 'PROJEÇÃO MÊS'] = df_print_vo.loc[i, '% META MÊS'] * round(float(sum(list(df_print_vo['META']))), 2)

df_print_vo = df_print_vo.rename(columns={'DT_EMISSAO': 'Data',
                                          'META': 'Meta',
                                          'VENDA BRUTA': 'Venda SIAC',
                                          'VENDA LÍQUIDA': 'Venda - Devolução',
                                          'DEVOLUÇÃO': 'Devolução',
                                          '% DEV': '% Dev',
                                          'VENDA_CNPJ': 'Venda CNPJ',
                                          'CNPJ': 'Clientes CNPJ',
                                          '% DEV CNPJ': '% Dev CNPJ',
                                          'VENDA_CPF': 'Venda CPF',
                                          'CPF': 'Clientes CPF',
                                          '% DEV CPF': '% Dev CPF',
                                          '% META DIA': '% Dia',
                                          '% META MÊS': '% Mês',
                                          'PROJEÇÃO MÊS': 'Projeção do mês'})





background_color = colors(df_print_vo, df_print_vo['Data'])
line_table_color = colors(df_print_vo, df_print_vo['Data'])
font_color = font_color_condicioning(df_print_vo, df_print_vo['Data'])





percent_total_dev_vo = (round(float(sum(list(df_print_vo['Devolução']))), 2) / round(float(sum(list(df_print_vo['Venda SIAC']))), 2))
percent_total_dev_pj_vo = (round(float(sum(list(df_print_vo['DEVOL_CNPJ']))), 2) / round(float(sum(list(df_print_vo['Venda CNPJ']))), 2))
percent_total_dev_pf_vo = (round(float(sum(list(df_print_vo['DEVOL_CPF']))), 2) / round(float(sum(list(df_print_vo['Venda CPF']))), 2))


df_print_vo.loc[len(df_print_vo), 'Data'] = ' '


df_print_vo = df_print_vo.fillna(0)

df_print_vo.loc[len(df_print_vo) - 1, 'Meta'] = round(float(sum(list(df_print_vo['Meta']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, 'Venda SIAC'] = round(float(sum(list(df_print_vo['Venda SIAC']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, 'Venda - Devolução'] = round(float(sum(list(df_print_vo['Venda - Devolução']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, 'Devolução'] = round(float(sum(list(df_print_vo['Devolução']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, '% Dev'] = percent_total_dev_vo
df_print_vo.loc[len(df_print_vo) - 1, 'Venda CNPJ'] = round(float(sum(list(df_print_vo['Venda CNPJ']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, 'Clientes CNPJ'] = round(float(sum(list(df_print_vo['Clientes CNPJ']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, '% Dev CNPJ'] = percent_total_dev_pj_vo
df_print_vo.loc[len(df_print_vo) - 1, 'Venda CPF'] = round(float(sum(list(df_print_vo['Venda CPF']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, 'Clientes CPF'] = round(float(sum(list(df_print_vo['Clientes CPF']))), 2)
df_print_vo.loc[len(df_print_vo) - 1, '% Dev CPF'] = percent_total_dev_pf_vo
df_print_vo.loc[len(df_print_vo) - 1, '% Dia'] = df_print_vo.loc[len(df_print_vo) - 1, 'Venda - Devolução'] / df_print_vo.loc[len(df_print_vo) - 1, 'Meta']
df_print_vo.loc[len(df_print_vo) - 1, '% Mês'] = df_print_vo.loc[len(df_print_vo) - 1, 'Venda - Devolução'] / df_print_vo.loc[len(df_print_vo) - 1, 'Meta']
df_print_vo.loc[len(df_print_vo) - 1, 'Projeção do mês'] = 0







df_print_vo = df_print_vo[['Data',
                           'Meta',
                           'Venda SIAC',
                           'Venda - Devolução',
                           'Devolução',
                           '% Dev',
                           'Venda CNPJ',
                           'Clientes CNPJ',
                           '% Dev CNPJ',
                           'Venda CPF',
                           'Clientes CPF',
                           '% Dev CPF',
                           '% Dia',
                           '% Mês',
                           'Projeção do mês']]



for i in df_print_vo.index:
    df_print_vo.loc[i, 'Meta'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Meta']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Venda SIAC'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Venda SIAC']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Devolução'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Devolução']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, '% Dev'] = '{:_.2%}'.format(df_print_vo.loc[i, '% Dev']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Venda CNPJ'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Venda CNPJ']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, '% Dev CNPJ'] = '{:_.2%}'.format(df_print_vo.loc[i, '% Dev CNPJ']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Clientes CNPJ'] = '{:_.0f}'.format(df_print_vo.loc[i, 'Clientes CNPJ']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Venda CPF'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Venda CPF']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, '% Dev CPF'] = '{:_.2%}'.format(df_print_vo.loc[i, '% Dev CPF']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Clientes CPF'] = '{:_.0f}'.format(df_print_vo.loc[i, 'Clientes CPF']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, '% Dia'] = '{:_.2%}'.format(df_print_vo.loc[i, '% Dia']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, '% Mês'] = '{:_.2%}'.format(df_print_vo.loc[i, '% Mês']).replace('.',',').replace('_','.')
    df_print_vo.loc[i, 'Projeção do mês'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Projeção do mês']).replace('.',',').replace('_','.')

    if df_print_vo.loc[i, 'Venda - Devolução'] < 0:
        df_print_vo.loc[i, 'Venda - Devolução'] = '-R$ {:_.2f}'.format(df_print_vo.loc[i, 'Venda - Devolução'] * -1).replace('.',',').replace('_','.')
    else:
        df_print_vo.loc[i, 'Venda - Devolução'] = 'R$ {:_.2f}'.format(df_print_vo.loc[i, 'Venda - Devolução']).replace('.',',').replace('_','.')

    if df_print_vo.loc[i, 'Data'] == ' ':
        df_print_vo.loc[i, 'Data'] = 'Total'
    else:
        df_print_vo.loc[i, 'Data'] = df_print_vo.loc[i, 'Data'].strftime('%d/%m/%Y %A')



df_print_vo = df_print_vo.replace('inf%', '').replace('-inf%', '').replace('0,00%', '').replace('R$ 0,00', '').replace('0', '')


table_vo = go.Figure(go.Table(header={'values':list(df_print_vo.columns),
                                      'fill_color': ['#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF'],
                                      'line_color': ['#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#F7F7F7',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF',
                                                     '#FFFFFF'],
                                      'align': ['center'],
                                      'font_color': ['#000000',
                                                     '#000000',
                                                     '#118DFF',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#FF0000',
                                                     '#E04392',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#E04392',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#000000',
                                                     '#000000',
                                                     '#000000']
                                      },
                                      
                               cells={'values':[list(df_print_vo['Data']),
                                                list(df_print_vo['Meta']),
                                                list(df_print_vo['Venda SIAC']),
                                                list(df_print_vo['Venda - Devolução']),
                                                list(df_print_vo['Devolução']),
                                                list(df_print_vo['% Dev']),
                                                list(df_print_vo['Venda CNPJ']),
                                                list(df_print_vo['Clientes CNPJ']),
                                                list(df_print_vo['% Dev CNPJ']),
                                                list(df_print_vo['Venda CPF']),
                                                list(df_print_vo['Clientes CPF']),
                                                list(df_print_vo['% Dev CPF']),
                                                list(df_print_vo['% Dia']),
                                                list(df_print_vo['% Mês']),
                                                list(df_print_vo['Projeção do mês'])],
                                      'fill_color': background_color,
                                      'line_color': line_table_color,
                                      'align': ['left', 'center'],
                                      'font_color': ['#000000',
                                                     '#000000',
                                                     '#118DFF',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#FF0000',
                                                     '#E04392',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#E04392',
                                                     '#000000',
                                                     '#FF0000',
                                                     '#000000',
                                                     '#000000',
                                                     '#000000']
                                      },

                              domain={'y': [0, 1]
                                      },

                         columnwidth=[proportion_size(pixels=_pixels, size=1500),
                                      proportion_size(pixels=_pixels, size=1000),
                                      proportion_size(pixels=_pixels, size=1000),
                                      proportion_size(pixels=_pixels, size=1000),
                                      proportion_size(pixels=_pixels, size=800),
                                      proportion_size(pixels=_pixels, size=400),
                                      proportion_size(pixels=_pixels, size=800),
                                      proportion_size(pixels=_pixels, size=400),
                                      proportion_size(pixels=_pixels, size=550),
                                      proportion_size(pixels=_pixels, size=800),
                                      proportion_size(pixels=_pixels, size=400),
                                      proportion_size(pixels=_pixels, size=550),
                                      proportion_size(pixels=_pixels, size=400),
                                      proportion_size(pixels=_pixels, size=400),
                                      proportion_size(pixels=_pixels, size=1000)]
                            )
)

table_vo = table_vo.update_layout(

    #Title config ---------------------------------------------------------------------------------------------------------------------#
    title  = {'text': f'▬'*int(10) + ' ' + lista_lojas[6][1][4:5] + lista_lojas[6][1][5:].lower() + ' ' + f'▬'*int(10),
              'font_color': '#000000',
              'x': 0.5,
              'y': 0.98,
              'font_size': proportion_size(pixels=_pixels, size=40)
              },

    #Size plot config -----------------------------------------------------------------------------------------------------------------#
    width  = pixels_width,
    height = pixels_height
)

table_vo.show()

# 

In [776]:
def colors(dataframe, datetime):
    
    colors = []
    date_colors = []
    destak_color = []
    

    for data in datetime:
        if data.strftime('%A') == 'Sunday':
            date_colors.append('#DEEFFF')
            destak_color.append('#DEEFFF')

        elif data.strftime('%d/%m/%Y') == dt.datetime.now().strftime('%d/%m/%Y'):
            date_colors.append('#F7F7F7')
            destak_color.append('#F7F7F7')
            
        else:
            date_colors.append('#FFFFFF')
            destak_color.append('#F7F7F7')


    for coluna in dataframe.columns:
        colors.append(date_colors)

    colors[2] = destak_color
    colors[3] = destak_color
    colors[6] = destak_color
    colors[9] = destak_color

    return colors

In [783]:
def font_color_condicioning(dataframe, datetime):
    
    date_colors = ['#000000',
                   '#000000',
                   '#118DFF',
                   '#000000',
                   '#FF0000',
                   '#FF0000',
                   '#E04392',
                   '#000000',
                   '#FF0000',
                   '#E04392',
                   '#000000',
                   '#FF0000',
                   '#000000',
                   '#000000',
                   '#000000']
    
    colors = []

    for data in datetime:
        if int(data.strftime('%d')) > int(dt.datetime.now().strftime('%d')):
            colors.append([['#FFFFFF'] * len(dataframe.columns)])
        else:
            colors.append(date_colors)
            

    return colors

In [ ]:
['#000000', '#000000', '#118DFF', '#000000', '#FF0000', '#FF0000', '#E04392', '#000000', '#FF0000', '#E04392', '#000000', '#FF0000', '#000000', '#000000', '#000000']